***Note: this is the EcoliCoreModel.ipynb notebook. The
PDF version "The Escherichia coli Core Model: Bond Graph Analysis"
is available [here](EcoliCoreModel.pdf).***

# Introduction

The bond graph approach 
<cite data-cite="OstPerKat71,OstPerKat73,GawCra14,GawCurCra15,GawCra16,GawCra17"></cite> 
to modelling biomolecular systems of interest to systems biologists developed independently from the stoichiometric approach 
<cite data-cite="Pal06,Pal11,Pal15"></cite>.

However, the conceptual point of intersection of the two approaches is the fact that the stoichiometric matrix is the modulus of the conceptual multiport transformer linking reactions to species.
This was pointed out by <cite data-cite="CelGre09"></cite>. This means that the two approaches are complementary and each can build on the strengths of the other.

This notebook focuses on building modular models of metabolism and consequent pathway analysis; energetic issues are not explicitly considered here although they are implicit in the bond graph models.


## Strengths of the stoichiometric approach

1. The stoichiometric matrix provides a precise description of the structure of chemical reaction networks.
2. Stoichiometric matrices are available for biologically significant systems including whole-cell models.
3. Powerful linear algebra concepts such as matrix null spaces and singular-value decompositions can be applied.
4. It provides the basis for flux-balance analysis <cite data-cite="OrtThiPal10"></cite>.

## Strengths of the bond graph approach
1. Bond graph provide an energy-based approach to modelling.
2. The bond graph model can be analysed for energy flows and efficiency as described by
<cite data-cite="GawCra18"></cite>.
3. The bond graph approach is multi domain and can thus, for example, model electrochemical systems including neurodynamics <cite data-cite="GawSieKam17"></cite> and redox reactions <cite data-cite="Gaw17a"</cite>.
4. The bond graph approch is modular: subsystems can be connected using energy ports <cite data-cite="GawCurCra15,GawCra16"></cite>.
5. The chemostat concept <cite data-cite="GawCra16"></cite> gives a convenient and flexible way of turning a closed system into an open system and analysing the concomitant pathway structure.
6. BondGraphTools provide a symbolic basis for describing and analysing bondgraphs within Python.

## Import some python code
The bond graph analysis uses a number of Python modules:

In [1]:
import importlib as imp
import Extract
import stoich as st
imp.reload(st)
## Stoich conversion
import stoichBondGraph as stbg
#imp.reload(stbg)
import numpy as np
import BondGraphTools as bgt
## Display (eg disp.SVG(), disp.
import IPython.display as disp
from timeit import default_timer as timer

## Combining the two approaches
The key stoichiometric concept of pathways has already been given a bond graph interpretation <cite data-cite="Gaw17,GawCra17"</cite>.


This note shows how:

1. bond graphs can be created from stoichiometric data using the ecoli core model <cite data-cite="OrtFlePal10"></cite> as an example.
2. bond graphs of subsystems can be extracted from such stoichiometric data
3. pathways can be anaysed and behavior elucidated using appropriate chice of chemostats
4. bond graphs of the extracted subsystems can be recombined in a modular fashion

Much remains to be done in exploiting the combination of the two approaches including:

1. A new look at flux-balance analysis.
2. Further work on energy and efficiency of cellular systems.
3. Using the large stoichiometric models available either directly or extracted from models in SBML or cellML format.

## Missing electrons

It seems that some stoichiometric models (for example the ecoli core model 
<cite data-cite="OrtThiPal10"></cite>
) 
do not contain information about charges in general and electrons in particular. There appears to be an ambiguity in that that species H  sometimes means \ch{H+} and sometimes \ch{H <> H+ + e-}. 

## External metabolites
The standard stoichiometric approach is to create open systems from closed systems by adding "dangling reactions" to species which connect to the outside world as external metabolites-- for example: \ch{ATP <>}. In contrast, the bond graph approacj would declare \ch{ATP} to be a chemostat. Thus when extracting a bond graph from a stoichiometric model, dangling reactions are deleted and the corresponding species added to a list of chemostats.

Chemostats provide a more flexible approach as they can be created without changing system structure.

# Extract various modules from the ecoli core model

## Extract full ecoli core model
In this case the ecoli core model is extracted from a spreadsheet describing the model: ecoli_core_model.xlsx.
This is modified to remove the "biomass" column and an integer version of the stoichiometric matrix, together with reactions and species is produced.

In [2]:
imp.reload(Extract)
sm_real = Extract.extract(quiet=True)
sm = Extract.Integer(sm_real)
sm['name'] = 'ecoli_core_model_abg'

Multiplying reaction CYTBD ( 14 ) by 2.0 to avoid non-integer species O2 ( 55 )


### Create a bond graph from the stoichiometric matrix, reactions and species.

In [3]:
stbg.model(sm)
import ecoli_core_model_abg

## Extract TCA part of full model 

A list of the reactions of the TCA cycle is provided and the corresponding species and stoichiometric matrix are extracted. Note that the NAD/NADP interconversion reaction NADTRHD is included.

In [4]:
reaction = ['CS','ACONTA','ACONTB','ICDHYR','AKGDH','SUCOAS',
               'FRD7',
                'SUCDI','FUM','MDH',
        'NADTRHD']
s0 = Extract.choose(sm,reaction=reaction)
disp.Latex(st.sprintrl(s0,chemformula=True))

<IPython.core.display.Latex object>

### Create bond graph and recreate stoichiometry

The bond graph TCA_abg is created and written to TCA_abg.py from whence it can me imported. The stoichiometric matrix generated from TCA_abg.model() is compared with that extracted from the ecoli core model to check that all is working correctely.

In [5]:
s0['name'] = 'TCA_abg'
stbg.model(s0)
import TCA_abg
imp.reload(TCA_abg)
s = st.stoich(TCA_abg.model(),quiet=True)
err = np.linalg.norm(s['N']-s0['N'])
print("Error:",err)

Error: 0.0


In [6]:
#bgt.draw(TCA_abg.model())

### Pathway analysis
Thee species corresponding to ATP hydrolysis, NAD/NADH, NADP/NADPH and Q8/Q8H2 (ubiquone, but maybe FAD, I think) are set as the basic chemostats in the list chemostats0. 

The  overall reaction of the TCA cycle converts ACCOA to COA and \ch{CO2} 
<cite data-cite="GarGri17"></cite>, so these three species are also set as chemostats.


In [7]:
print(st.sprintp(s))
chemostats0 = ['ADP', 'ATP', 'H2O', 'NAD', 'NADH', 'PI','H'
              ,'Q8','Q8H2']
sc0 = st.statify(s,chemostats=chemostats0)
print(st.sprintp(sc0))
chemostats = ['ACCOA','COA','CO2']
chemostats.extend(chemostats0)
sc = st.statify(s,chemostats=chemostats)
print(st.sprintp(sc))


1 pathways
0:  + FRD7 + SUCDI

1 pathways
0:  + FRD7 + SUCDI

2 pathways
0:  + FRD7 + SUCDI
1:  + CS + ACONTA + ACONTB + ICDHYR + AKGDH - SUCOAS - FRD7 + FUM + MDH + NADTRHD



With only the basic chemostats, there is one internal cycle. This performs no conversions and so the reaction is empty.

In [8]:
sp0 = st.path(s,sc0)
disp.Latex(st.sprintrl(sp0,chemformula=True,split=10))

<IPython.core.display.Latex object>

When the chemostats ['ACCOA','COA','CO2'] are added, the entire TCA cycle appears as a pathway and the overall reaction is

In [9]:
sp = st.path(s,sc)
disp.Latex(st.sprintrl(sp,chemformula=True,split=10))

<IPython.core.display.Latex object>

Pathway reactions:

1. A null cycle (shared with closed system)
2. Pathway from ACCOA to CO2

## Include Pyruvate (Pyr) reactions: PDH and PFL
To take this example further, include the two reactions converting pyruvate to ACCOA.

In [10]:
reaction.extend(['PDH','PFL'])
s0 = Extract.choose(sm,reaction=reaction)
disp.Latex(st.sprintrl(s0,chemformula=True))

<IPython.core.display.Latex object>

### Create bond graph and recreate stoichiometry

In [11]:
s0['name'] = 'PyrTCA_abg'
stbg.model(s0)
import PyrTCA_abg
imp.reload(PyrTCA_abg)
s = st.stoich(PyrTCA_abg.model(),quiet=True)
err = np.linalg.norm(s['N']-s0['N'])
print("Error:",err)

Error: 0.0


### Pathway analysis
The relevant chemostats are now the substrate \ch{PYR} and the product \ch{CO2} together with the reactions ATP hydrolysis, NAD/NADH, NADP/NADPH and Q8/Q8H2 nf the additional product FOR

In [12]:
print(st.sprintp(s))
chemostats0 = ['ADP', 'ATP', 'H2O', 'NAD', 'NADH', 'PI','NADP', 'NADPH','H'
              ,'Q8','Q8H2','FOR']
sc0 = st.statify(s,chemostats=chemostats0)
print(st.sprintp(sc0))
chemostats = ['PYR','CO2']
chemostats.extend(chemostats0)
sc = st.statify(s,chemostats=chemostats)
print(st.sprintp(sc))


1 pathways
0:  + FRD7 + SUCDI

2 pathways
0:  + FRD7 + SUCDI
1:  + NADTRHD

4 pathways
0:  + FRD7 + SUCDI
1:  + NADTRHD
2:  + CS + ACONTA + ACONTB + ICDHYR + AKGDH - SUCOAS - FRD7 + FUM + MDH + PDH
3:  + CS + ACONTA + ACONTB + ICDHYR + AKGDH - SUCOAS - FRD7 + FUM + MDH + PFL



In [13]:
sp0 = st.path(s,sc0)
disp.Latex(st.sprintrl(sp0,chemformula=True,split=10))

<IPython.core.display.Latex object>

In [14]:
sp = st.path(s,sc)
disp.Latex(st.sprintrl(sp,chemformula=True,split=10))

<IPython.core.display.Latex object>

Pathway reactions:

1. A null cycle (shared with closed system)
2. Pathway from PYR to CO2 via reaction PDH
3. Pathway from PYR to CO2 via reaction PFL

## Extract the Electron Transport Chain (ETC )

In [15]:
reaction = ['NADH16','CYTBD']
s0 = Extract.choose(sm,reaction=reaction)
disp.Latex(st.sprintrl(s0,chemformula=True))

<IPython.core.display.Latex object>

In [16]:
s0['name'] = 'ETC_abg'
stbg.model(s0)
import ETC_abg
imp.reload(ETC_abg)
s = st.stoich(ETC_abg.model(),quiet=True)
err = np.linalg.norm(s['N']-s0['N'])
print("Error:",err)
s['reaction'] = s0['reaction']


Error: 0.0


### Pathway analysis

In [17]:
print(st.sprintp(s))
chemostats = ['NADH','NAD','O2','H2O','H','H_E']
sc = st.statify(s,chemostats=chemostats)
print(st.sprintp(sc))


0 pathways

1 pathways
0:  + 2 NADH16 + CYTBD



In [18]:
sp = st.path(s,sc)
disp.Latex(st.sprintrl(sp,chemformula=True))

<IPython.core.display.Latex object>

## Extract ATPase

In [19]:
reaction = ['ATPS4R']
s0 = Extract.choose(sm,reaction=reaction)
disp.Latex(st.sprintrl(s0,chemformula=True))

<IPython.core.display.Latex object>

In [20]:
s0['name'] = 'ATP_abg'
stbg.model(s0)
import ATP_abg
imp.reload(ATP_abg)
s = st.stoich(ATP_abg.model(),quiet=True)
err = np.linalg.norm(s['N']-s0['N'])
print("Error:",err)
s['reaction'] = s0['reaction']


Error: 0.0


### Pathway analysis

In [21]:
print(st.sprintp(s))
chemostats = ['H','H_E','ATP','ADP','PI','H2O']
sc = st.statify(s,chemostats=chemostats)
print(st.sprintp(sc,removeSingle=False))


0 pathways

1 pathways
0:  + ATPS4R



In [22]:
sp = st.path(s,sc,removeSingle=False)
disp.Latex(st.sprintrl(sp,chemformula=True))

<IPython.core.display.Latex object>

## Extract Glycolysis

In [23]:
reaction = ['GLCPTS','PGI','PFK','FBP','FBA','TPI','GAPD','PGK','PGM','ENO','PYK']
s0 = Extract.choose(sm,reaction=reaction)
disp.Latex(st.sprintrl(s0,chemformula=True))


<IPython.core.display.Latex object>

In [24]:
s0['name'] = 'GLY_abg'
stbg.model(s0)
import GLY_abg
imp.reload(GLY_abg)
s = st.stoich(GLY_abg.model(),quiet=True)
err = np.linalg.norm(s['N']-s0['N'])
print("Error:",err)

Error: 0.0


### Pathway analysis

In [25]:
chemostats0 = ['ADP','ATP','H2O','PI','H']
sc0 = st.statify(s,chemostats=chemostats0)
chemostats = ['GLCD_E','PYR','NAD','NADH']
chemostats.extend(chemostats0)
sc = st.statify(s,chemostats=chemostats)
print(st.sprintp(sc))
sp = st.path(s,sc)
disp.Latex(st.sprintrl(sp,chemformula=True))

2 pathways
0:  + PFK + FBP
1:  + GLCPTS + PGI + PFK + FBA + TPI + 2 GAPD - 2 PGK - 2 PGM + 2 ENO + PYK



<IPython.core.display.Latex object>

# Modularity: the Glycolysis/TCA network
To examine the glycolysis/TCA network, it is convenient and informative to take a modular approach:  the glycolysis network and the TCA network are extracted as separate subsystems and then combined. The example shows two approaches to combining the subsystems:

1. combining the stoichiometric matrices
2. combining the bond graphs.

## Create composite model

In [26]:
GlyTCA = bgt.new(name='GlyTCA')   # Create system
Gly = GLY_abg.model()
TCA = PyrTCA_abg.model()
GlyTCA.add(Gly,TCA)

### Unify common species - stoichiometric approach
PYR is produced by Gly and consumed by TCA.
ATP, ADP, PI, H, NAD, NADH , H2O are common to both modules.

In [27]:
imp.reload(st)
common = ['PYR','ATP','ADP','PI','H','NAD','NADH','H2O']
st.unify(s,commonSpecies=common)

### Pathway analysis

In [28]:
chemostats0 = ['ADP', 'ATP', 'H2O', 'NAD', 'NADH', 'PI','H'
              ,'Q8','Q8H2','FOR']
chemostats = ['GLCD_E','CO2']
chemostats.extend(chemostats0)
sc = st.statify(s,chemostats=chemostats)
print(st.sprintp(sc))

Chemostat CO2 is not a model species
Chemostat Q8 is not a model species
Chemostat Q8H2 is not a model species
Chemostat FOR is not a model species
1 pathways
0:  + PFK + FBP



In [29]:
sp = st.path(s,sc)
disp.Latex(st.sprintrl(sp,chemformula=True))

<IPython.core.display.Latex object>

Pathway reactions:

1. A futile cycle
2. A null cycle
3. Pathway from GLCD_E to CO2 via reaction PDH
4. Pathway from GLCD_E to CO2 via reaction PFL

### Unify common species - bond graph  approach
This version creates a bond graph model of the unified system by explicitly replacing C components by ports in the subsystems, creating new C components and connecting via 0 junctions and ports.

In [30]:
import modular
modular.unify(GlyTCA,common=common,quiet=True)

In [31]:
##bgt.draw(GlyTCA)

#### Pathway analysis

In [32]:
s = st.stoich(GlyTCA,quiet=True)
disp.Latex(st.sprintrl(s,chemformula=True))

<IPython.core.display.Latex object>

In [33]:
sc = st.statify(s,chemostats=chemostats)
print(st.sprintp(sc))

4 pathways
0:  + PFK + FBP
1:  + FRD7 + SUCDI
2:  + GLCPTS + PGI + PFK + FBA + TPI + 2 GAPD - 2 PGK - 2 PGM + 2 ENO + PYK + 2 CS + 2 ACONTA + 2 ACONTB + 2 ICDHYR + 2 AKGDH - 2 SUCOAS - 2 FRD7 + 2 FUM + 2 MDH + 2 NADTRHD + 2 PDH
3:  + GLCPTS + PGI + PFK + FBA + TPI + 2 GAPD - 2 PGK - 2 PGM + 2 ENO + PYK + 2 CS + 2 ACONTA + 2 ACONTB + 2 ICDHYR + 2 AKGDH - 2 SUCOAS - 2 FRD7 + 2 FUM + 2 MDH + 2 NADTRHD + 2 PFL



In [34]:
sp = st.path(s,sc)
disp.Latex(st.sprintrl(sp,chemformula=True))

<IPython.core.display.Latex object>

Pathway reactions are the same as before:

1. A futile cycle
2. A null cycle
3. Pathway from GLCD_E to CO2 via reaction PDH
4. Pathway from GLCD_E to CO2 via reaction PFL

# Modularity: metabolism
The above methods are brought together to generate the metabolic pathways by combining the modules describing:
Glycolysis, the TCA cycle, the Electron Transport Chain and ATPase. Once again, the overall bond graph is created  by comineing moduels and unifying common species.

In [35]:
Met = bgt.new(name='Met')   # Create system
Gly = GLY_abg.model()
TCA = PyrTCA_abg.model()
ETC = ETC_abg.model()
ATP = ATP_abg.model()
Met.add(Gly,TCA,ETC,ATP)

In [36]:
## Unify species common to modules
common = ['PYR','ATP','ADP','PI','H','H_E','NAD','NADH','H2O','Q8','Q8H2']
modular.unify(Met,common=common,quiet=True)

## Pathway analysis

In [37]:
s = st.stoich(Met,quiet=True)
disp.Latex(st.sprintrl(s,chemformula=True))

<IPython.core.display.Latex object>

In [38]:
imp.reload(st)
chemostats0 = ['ADP','ATP','H2O','PI','H']
chemostats = ['GLCD_E','CO2','O2']
chemostats.extend(chemostats0)
print(chemostats)
sc = st.statify(s,chemostats=chemostats)
print(st.sprintp(sc))

['GLCD_E', 'CO2', 'O2', 'ADP', 'ATP', 'H2O', 'PI', 'H']
3 pathways
0:  + PFK + FBP
1:  + FRD7 + SUCDI
2:  + 2 GLCPTS + 2 PGI + 2 PFK + 2 FBA + 2 TPI + 4 GAPD - 4 PGK - 4 PGM + 4 ENO + 2 PYK + 4 CS + 4 ACONTA + 4 ACONTB + 4 ICDHYR + 4 AKGDH - 4 SUCOAS - 4 FRD7 + 4 FUM + 4 MDH + 4 NADTRHD + 4 PDH + 20 NADH16 + 12 CYTBD + 27 ATPS4R



In [39]:
sp = st.path(s,sc)
disp.Latex(st.sprintrl(sp,chemformula=True))

<IPython.core.display.Latex object>

There are three pathways: 

1. A futile cycle
2. A null cycle
3. Each GLCD_E, combined with \ch{6 O2}, reverses ATP hydrolysis  (\ch{ATP + H2O <> ADP + PI + H+}) to give 17.5 ATP molecules with an additional \ch{6 H2O} and \ch{6 CO2}; this is the value quoted
by <cite data-cite={Pal15}>Palsson (2015)</cite>.